In [1]:
!nvidia-smi

Sun May 15 05:58:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [3]:
! pip install -U synonyms

     |████████████████████████████████| 10.7 MB 27.8 MB/s 
  Created wheel for synonyms: filename=synonyms-3.16.0-py3-none-any.whl size=10832785 sha256=ef2a2ba06c79feab723ce78d8700e503ec8531220a57db17b1ec2c6ca686f2c7
  Stored in directory: /root/.cache/pip/wheels/e4/cd/43/b4548753509a94471fc946967a07116252d49aeeb689db8f7c
Successfully built synonyms


In [4]:
! pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 19 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.


In [5]:
! pip install transformers==4.0.1

     |████████████████████████████████| 1.4 MB 25.8 MB/s 
     |████████████████████████████████| 880 kB 66.5 MB/s 
     |████████████████████████████████| 2.9 MB 31.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=cd9f77627587a5c9b8a46bbb3bf99d6545d1a7f997d98d70836be71a91b0e1db
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [6]:
import synonyms

[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/usr/local/lib/python3.7/dist-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/usr/local/lib/python3.7/dist-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz] ...

[Synonyms] downloading data from https://github.com/chatopera/Synonyms/releases/download/3.15.0/words.vector.gz to /usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz ... 
 this only happens if SYNONYMS_WORD2VEC_BIN_URL_ZH_CN is not present and Synonyms initialization for the first time. 
 It would take minutes that depends on network.

[Synonyms] downloaded.



In [7]:
synonyms.nearby('篮球')

(['篮球', '橄榄球', '排球', '棒球', '美式足球', '冰球', '拳击', '网球', '高尔夫球', '高球'],
 [1.0,
  0.81482756,
  0.78554475,
  0.7845952,
  0.7815255,
  0.7550466,
  0.7411452,
  0.7350726,
  0.72586256,
  0.7199612])

# EDA(Easy Data Augmentation)
![UDA1](https://img-blog.csdnimg.cn/c5ffcca4482c4c42beb6f1215e37657c.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
### EDA：用于提高文本分类任务性能的简单数据增强技术。 EDA 由四个简单但功能强大的操作组成：同义词替换、随机插入、随机交换和随机删除。
### 之前的工作已经提出了一些 NLP 中数据增强的技术，回译通过将句子翻译成法语然后再翻译成英语来生成新数据。

### 对于训练集中的给定句子，随机选择并执行以下操作之一：
* 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
* 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
* 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
* 随机删除（RD）：以概率 p 随机删除句子中的每个单词。

### 停用词 stop word

停用词是指在信息检索中，为节省存储空间和提高搜索效率，在处理自然语言数据（或文本）之前或之后会自动过滤掉某些字或词，这些字或词即被称为Stop Words（停用词）
```
str = "00000003210Runoob01230000000"; 
print str.strip( '0' );  # 去除首尾字符 0

str2 = "   Runoob      ";   # 去除首尾空格
print str2.strip();

3210Runoob0123
Runoob
```


In [8]:
# strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
stop_words = {word.strip() for word in open('/content/drive/MyDrive/Colab Notebooks/dataset/baidu_stopwords.txt', 'r', encoding='utf8').readlines()}

In [9]:
# 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
import random
def get_synonyms(word):
  # (['mama'],['0.9'])
  #取出元组第0个元素（['mama']），并去重
  sys = set(synonyms.nearby(word)[0])
  #将word从同义词列表中去除
  if word in sys:
    sys.remove(word)
  return list(sys)
  # 如果输入"给力" 可能没有同义词（同义词只有他自己） 则返回  ([],[])


def synonym_replacement(words, n):
  new_words = words.copy()
  # 遍历句子中的每个词，并且这个词不是停用词
  # set()去重， 以列表形式返回
  random_word_list = list(set([word for word in words if word not in stop_words]))
  # 打乱
  random.shuffle(random_word_list)
  num_replaced = 0
  for random_word in random_word_list:
    # 得到近义词列表
    synonyms = get_synonyms(random_word)
    if len(synonyms) >= 1:
      # 随机取出一个同义词
      synonym = random.choice(list(synonyms))
      # 用synonym替换原词
      new_words = [synonym if word == random_word else word for word in new_words]
      num_replaced += 1
    if num_replaced >= n:
      break
  # new_words 已经是个列表了 下一块代码举例
  sentence = ' '.join(new_words)
  new_words = sentence.split(' ')
  return new_words

### 存在一种情况  
*有近义词：**actor** -> **film star** 一个单词的近义词是两个单词*

sentence = ['in', 'actor']

*默认new_words = ['in', 'film star']*

*希望有如下表示：*
new_words = ['in', 'film', 'star']

In [10]:
# 随机删除（RD）：以概率 p 随机删除句子中的每个单词。
def random_deletion(words, p):
  if len(words) == 1:
    return words
  new_words = []
  for word in words:
    # 概率
    r = random.uniform(0, 1)
    # r>p 保留
    if r > p:
      new_words.append(word)
  # 如果都删没了 随机返回一个词
  if len(new_words) == 0:
    random_int = random.randint(0, len(words) - 1)
    return [words[random_int]]
  return new_words

In [11]:
# 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
def swap_word(new_words):
  random_idx_1 = random.randint(0, len(new_words) - 1)
  random_idx_2 = random_idx_1
  
  count = 0
  # 两者相等重新取idx_2
  while random_idx_2 == random_idx_1:
    random_idx_2 - random.randint(0, len(new_words) - 1)
    count += 1
    # 取了三次还是相等 立即推
    if count > 3:
      return new_words

  new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
  return new_words

def random_swap(words, n):
  new_words = words.copy()
  for _ in range(n):
    new_words = swap_word(new_words)
  return new_words

In [12]:
# 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
def add_word(new_words):
  # 同义词列表
  synonyms = []
  count = 0
  while len(synonyms) < 1:
    # 在句子中随机找到一个词
    random_word = new_words[random.randint(0, len(new_words) - 1)]
    synonyms = get_synonyms(random_word)
    count += 1
    # 找了10次同义词数量仍小于1，立即推
    if count >= 10:
      return 
  #将同义词表中第一个插入
  random_synonym = synonyms[0]
  # 取出要插入的位置  
  random_idx = random.randint(0, len(new_words) - 1)
  new_words.insert(random_idx, random_synonym)

def random_insert(words, n):
  new_words = words.copy()
  for i in range(n):
    add_word(new_words)
  return new_words

In [13]:
words = synonyms.seg('目前华为部分型号的手机产品出现货少的现象')
print(words)
# 后面的是词性

(['目前', '华为', '部分', '型号', '的', '手机', '产品', '出现', '货', '少', '的', '现象'], ['t', 'nr', 'n', 'n', 'uj', 'n', 'n', 'v', 'n', 'a', 'uj', 'n'])


In [14]:
from random import shuffle
# EDA方法
# 参数num_aug：增加了几条数据
def eda(sentence, alpha_sr= 0.1, alpha_ri = 0.1, alpha_rs = 0.1, p_rd = 0.1, num_aug = 9):
  words = synonyms.seg(sentence)[0]
  num_words = len(words)
  augmented_sentence = []
  
  # 每种技术增加了多少样本
  num_new_per_tech = int(num_aug / 4) + 1
  # = 3

  # 同义词替换数量
  n_sr = max(1, int(alpha_sr * num_words))
  # 随机插入数量
  n_ri = max(1, int(alpha_ri * num_words))
  # 随机交换数量
  n_rs = max(1, int(alpha_rs * num_words))

  # 同义词替换
  for i in range(num_new_per_tech):
    a_words = synonym_replacement(words, n_sr)
    # a_words 是列表 []
    print('a-words(同义词替换):',a_words)
    augmented_sentence.append(' '.join(a_words))
  # 随机插入
  for i in range(num_new_per_tech):
    a_words = random_insert(words, n_ri)
    augmented_sentence.append(' '.join(a_words))
  # 随机交换
  for i in range(num_new_per_tech):
    a_words = random_swap(words, n_rs)
    augmented_sentence.append(' '.join(a_words))
  # 随机删除
  for i in range(num_new_per_tech):
    a_words = random_deletion(words, p_rd)
    augmented_sentence.append(' '.join(a_words))

  shuffle(augmented_sentence)

  if num_aug >= 1:
    augmented_sentence = augmented_sentence[:num_aug]
  else: # num_aug<1
    keep_prob = num_aug / len(augmented_sentence)
    # random_delete
    augmented_sentence = [s for s in augmented_sentence if random.uniform(0, 1) < keep_prob]

  return augmented_sentence



In [15]:
eda('9月15日以来，台积电、高通、三星等华为的重要合作伙伴，只要没有美国的相关许可证，都无法供应芯片给华为，而中芯国际等国产芯片企业，也因采用美国技术，而无法供货给华为。目前华为部分型号的手机产品出现货少的现象，若该形势持续下去，华为手机业务将遭受重创。')

a-words(同义词替换): ['9', '月前', '15', '日', '以来', '，', '台积电', '、', '高通', '、', 'LG', '等', 'OPPO', '的', '重要', '合作伙伴', '，', '只要', '没有', '哥伦比亚', '的', '相关', '许可证', '，', '都', '无法', '供应', '芯片', '给', 'OPPO', '，', '而', '中芯国际', '等', '国产', '芯片', '企业', '，', '也', '因', '采用', '哥伦比亚', '技术', '，', '而', '无法', '供货', '给', 'OPPO', '。', '目前', 'OPPO', '部分', '型号', '的', '手机', '品类', '出现', '货', '少', '的', '现象', '，', '若', '该', '形势', '急剧', '下去', '，', 'OPPO', '手机', '产品销售', '将', '遭受', '重创', '。']
a-words(同义词替换): ['9', '月', '15', '日', '以来', '，', '台积电', '、', 'NVIDIA', '、', '三星', '等', '华为', '的', '重要', '合作伙伴', '，', '只要', '没有', '美国', '的', '相关', '许可证', '，', '都', '无法', '供给量', '芯片', '给', '华为', '，', '而', '中芯国际', '等', '国产', '芯片', '企业', '，', '也', '因', '采用', '美国', '技术', '，', '而', '无法', '供货', '给', '华为', '。', '目前', '华为', '部分', '型号', '的', '手机', '家电产品', '出现', '货', '少', '的', '不良现象', '，', '若', '该', '情势', '持续', '下去', '，', '华为', '手机', '投资业务', '将', '遭致', '重创', '。']
a-words(同义词替换): ['9', '月', '15', '日', '以来', '，', 'GT5316SB0', '、', '德州仪器', '、', 

['9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， 台积电 高通 、 三星 华为 的 重要 ， 只要 没有 美国 的 相关 许可证 ， 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 该 形势 持续 下去 ， 华为 手机 业务 。',
 '9 月 15 日 以来 由于 ， 台积电 国际形势 、 高通 、 三星 等 华为 的 新手机 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 闪存 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 国产化 现象 ， 若 甚至于 该 形势 持续 下去 甚至于 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， 台积电 、 NVIDIA 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供给量 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 家电产品 出现 货 少 的 不良现象 ， 若 该 情势 持续 下去 ， 华为 手机 投资业务 将 遭致 重创 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象

## 闭包数据增强
数据集中每条数据有两个句子 \
a, b, 1\
a, c, 1\
a, d, 0\
a-b(相似), a-c => b-c\
a-b, ad不相似 => bd不相似

In [16]:
import pandas as pd
from tqdm import tqdm
import json
def parse_data(path, data_type='train'):
  sentence_a = []
  sentence_b = []
  labels = []

  with open(path, 'r', encoding = 'utf8') as f:
    for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
      line = json.loads(line)
      sentence_a.append(line['sentence1'])
      sentence_b.append(line['sentence2'])
      if data_type != 'test':
        labels.append(int(line['label']))
      else:
        labels.append(0)

  df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns = ['text_a', 'text_b', 'labels'])
  return df

In [17]:
train_df = parse_data('/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/train.json', data_type='train')

Reading train data: 100%|██████████| 34334/34334 [00:00<00:00, 122871.81it/s]


In [18]:
train_df.head()

,text_a,text_b,labels
0,蚂蚁借呗等额还款可以换成先息后本吗,借呗有先息到期还本吗,0
1,蚂蚁花呗说我违约一次,蚂蚁花呗违约行为是什么,0
2,帮我看一下本月花呗账单有没有结清,下月花呗账单,0
3,蚂蚁借呗多长时间综合评估一次,借呗得评估多久,0
4,我的花呗账单是***，还款怎么是***,我的花呗，月结出来说让我还***元，我自己算了一下详细名单我应该还***元,1


In [19]:
for data in train_df.groupby(by = ['text_a']):
  if len(data[1])>=3:
   
    print('数据类型:',type(data))
    print('数据长度:',len(data))
    print('data[0]:',data[0]) # text_a句子
    print('-------')
    print('data[1]:',data[1]) # 以text_a为主键返回的dataFrame
    print('-------')
  
    print('data[1].info():',data[1].info())
    break

数据类型: <class 'tuple'>
数据长度: 2
data[0]: 为什么我开通不了花呗
-------
data[1]:            text_a                            text_b  labels
21     为什么我开通不了花呗  我一直想买苹果***p，没钱，想分期付款，除了花呗，还有什么可以       0
1333   为什么我开通不了花呗                        为什么不可以开通花呗       0
17994  为什么我开通不了花呗                          我为何打不开花呗       0
18096  为什么我开通不了花呗                         我开通不了蚂蚁花呗       1
19484  为什么我开通不了花呗           你直接帮我查一下 确实开通不了花呗吗？我的账户       1
31930  为什么我开通不了花呗                         电脑端怎么开通花呗       0
-------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 21 to 31930
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text_a  6 non-null      object
 1   text_b  6 non-null      object
 2   labels  6 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 192.0+ bytes
data[1].info(): None


### 举例
a : 为什么我开通不了花呗 b: 我一直想买苹果 p，没钱，想分期付款，除了花呗，还有什么可以 c:我开通不了蚂蚁花呗 ab 不相似， ac相似，=》bc不相似

In [20]:
from collections import defaultdict
def aug_group_by_a(df):
  aug_data = defaultdict(list)
  # 以text_a中的句子为g
  for g, data in df.groupby(by = ['text_a']):
    # 如果只有一条数据 无法数据增强
    if len(data) < 2:
      continue
    # iloc[ : , : ] 行列切片以“，”隔开，前面的冒号就是取行数，后面的冒号是取列数
    for i in range(len(data)):
      for j in range(i+1, len(data)):
        # 取出b的值，a,b的label
        row_i_text = data.iloc[i, 1]
        row_i_label = data.iloc[i, 2]
        # 取出c的值，a,c的label
        row_j_text = data.iloc[j, 1]
        row_j_label = data.iloc[j, 2]
        # 如果 ab ， ac都不相似则不考虑
        if row_i_label == row_j_label == 0:
          continue
        # ab相似， ac也相似， bc就相似
        aug_label = 1 if row_i_label == row_j_label == 1 else 0

        aug_data['text_a'].append(row_i_text)
        aug_data['text_b'].append(row_j_text)
        aug_data['labels'].append(aug_label)
  return pd.DataFrame(aug_data)


In [21]:
aug_train_a = aug_group_by_a(train_df)
# 看增强了多少条数据
print(aug_train_a)

                               text_a                   text_b  labels
0    我一直想买苹果***p，没钱，想分期付款，除了花呗，还有什么可以                我开通不了蚂蚁花呗       0
1    我一直想买苹果***p，没钱，想分期付款，除了花呗，还有什么可以  你直接帮我查一下 确实开通不了花呗吗？我的账户       0
2                          为什么不可以开通花呗                我开通不了蚂蚁花呗       0
3                          为什么不可以开通花呗  你直接帮我查一下 确实开通不了花呗吗？我的账户       0
4                            我为何打不开花呗                我开通不了蚂蚁花呗       0
..                                ...                      ...     ...
223                           你是人工服务吗                   需求人工客服       0
224                           你是人工服务吗                   联系人工客服       1
225                            需求人工客服                   联系人工客服       0
226                     身份证过期可以用蚂蚁借呗吗           身份证过期可以注册蚂蚁借呗吗       0
227                            人工关闭花呗     一个帐户的花呗关，另一个帐户的花呗怎么开       0

[228 rows x 3 columns]


## UDA（Unsupervised Data Augmentation for Consistency Training）用于一致性训练的无监督数据增强
 ![UDA1](https://img-blog.csdnimg.cn/c9cb603261a1497c8093ee669b7923f2.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 什么是一致性训练？
### 数据增强 是 创建 逼近 真实的训练数据，并且不改变其标签
### 举例： a, b, 1  a c, 1 -> b, c, 1
### 一致性训练：增强前和增强后的标签 应该保持一致，利用这个特性训练

![unsupervised data augmentation](https://img-blog.csdnimg.cn/86f3aad1042b4bf381d662e9c3b48f0a.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

 半监督学习利用无标签样本增强模型鲁棒性
 * 给定输入x, 计算输出分布$p_{\theta}(y|x)$, 同时，给输入x进行数据增强，计算出分布$p_{\theta}(y|\hat x)$.
 * 给两个分布计算KL散度

![UDA5](https://img-blog.csdnimg.cn/9d10da70d1d0467e93ef5bb1267ac87f.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 解释上图
![code](https://img-blog.csdnimg.cn/d97f35fd41e0485185f40d50f4fd8e8d.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

**ps： unsup_x--(a,b)成为 no_grad_data,不需要反向传播** \
*sup:监督 unsup:无监督*

In [22]:
from bucket_sampler import SortedSampler, BucketBatchSampler
from EMA import *

In [23]:
import random
import numpy as np
import torch
config = {
        'train_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/train.json',
        'dev_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/dev.json',
        'test_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/test.json',
        'output_path': '.',
        'model_path': '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model',
        'batch_size': 16,
        'num_epochs': 1,
        'max_seq_len': 64,
        'learning_rate': 2e-5,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'unsup_data_ratio': 1.5,
        'uda_softmax_temp': 0.4,
        'uda_confidence_threshold': 0.8,
        'device': 'cuda',
        'n_gpus': 0,
        'logging_step': 300,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2022
    }

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

In [24]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [25]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids =True, return_attention_mask = True)

  inputs['input_ids'].append(inputs_dict['input_ids'])
  inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
  inputs['attention_mask'].append(inputs_dict['attention_mask'])
  inputs['labels'].append(label)

##  对偶数据增强
sentence1: a\
sentence2: b\
a, b, 1 变成 b, a, 1

In [26]:
# 无监督BERT输入
def build_unsup_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  # 左右
  lr_inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)
  # 右左
  rl_inputs_dict = tokenizer.encode_plus(sentence_b, sentence_a, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)

  # 元组的形式
  inputs['input_ids'].append((lr_inputs_dict['input_ids'], rl_inputs_dict['input_ids']))
  inputs['token_type_ids'].append((lr_inputs_dict['token_type_ids'], rl_inputs_dict['token_type_ids']))
  inputs['attention_mask'].append((lr_inputs_dict['attention_mask'], rl_inputs_dict['attention_mask']))
  inputs['labels'].append(label)


In [27]:
def read_data(config, tokenizer):
  train_df = parse_data(config['train_file_path'], data_type = 'train')
  dev_df = parse_data(config['dev_file_path'], data_type = 'dev')
  test_df = parse_data(config['test_file_path'], data_type = 'test')

  data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
  processed_data = {}
  unsup_data = defaultdict(list)
  for data_type, df in data_df.items():
    inputs = defaultdict(list)
    if data_type == 'train':
      reversed_inputs = defaultdict(list)

    for i, row in tqdm(df.iterrows(), desc=f'Preprocessing {data_type} data', total = len(df)):
      label = 0 if data_type == 'test' else row[2]
      sentence_a, sentence_b = row[0], row[1]
      build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)

      if data_type.startswith('test'):
        build_bert_inputs(inputs, label, sentence_b, sentence_a, tokenizer)


      build_unsup_bert_inputs(unsup_data, label, sentence_a, sentence_b, tokenizer)

    processed_data[data_type] = inputs
  processed_data['unsup_data'] = unsup_data
  return processed_data

# processed_data
# {
#    'train':,
#    'dev':,
#    'test':,
#    'unsup_data':   # 数据量最大的
# }




In [28]:
data = read_data(config, tokenizer)

Preprocessing test data: 100%|██████████| 3861/3861 [00:06<00:00, 586.35it/s]


In [29]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(AFQMCDataset, self).__init__()
    self.data_dict = data_dict

  def __getitem__(self, idx):
    data = (self.data_dict['input_ids'][idx],
         self.data_dict['token_type_ids'][idx],
         self.data_dict['attention_mask'][idx],
         self.data_dict['labels'][idx])
    return data
  
  def __len__(self):
    return len(self.data_dict['input_ids'])

In [30]:
class Collator:
  def __init__(self, max_seq_len, tokenizer):
    self.max_seq_len = max_seq_len
    self.tokenizer = tokenizer

  def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
    input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype = torch.long)
    token_type_ids = torch.zeros_like(input_ids)
    attention_mask = torch.zeros_like(input_ids)
    for i in range(len(input_ids_list)):
      seq_len = len(input_ids_list[i])
      if seq_len <= max_seq_len:
        input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype = torch.long)
        token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype = torch.long)
        attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype = torch.long)

      else:
        input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len - 1] + [self.tokenizer.sep_token_id], dtype = torch.long)
        token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype = torch.long)
        attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype = torch.long)

      labels = torch.tensor(labels_list, dtype = torch.long)
      return input_ids, token_type_ids, attention_mask, labels

  def __call__(self, examples):
    input_ids_list, token_type_ids_list, attention_mask_list, labels_list = list(zip(*examples))
    cur_max_seq_len = max(len(input_ids) for input_ids in input_ids_list)
    max_seq_len = min(cur_max_seq_len, self.max_seq_len)

    input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len)

    data_dict = {
        'input_ids':input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask':attention_mask,
        'labels':labels
    }
    return data_dict

In [31]:
collate_fn = Collator(config['max_seq_len'], tokenizer)

In [32]:
# UDA 无监督重新构造dataset 和 collator
class UnsupAFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(UnsupAFQMCDataset, self).__init__()
    self.data_dict = data_dict
  
  def __getitem__(self, idx):
    input_ids = self.data_dict['input_ids'][idx]
    token_type_ids = self.data_dict['token_type_ids'][idx]
    attention_mask = self.data_dict['attention_mask'][idx]
    labels = self.data_dict['labels'][idx]
    # input_ids[0]：lr_inputs_dict['input_ids']    (build_unsup_bert_inputs)
    # input_ids[1]：rl_inputs_dict['input_ids']
    return (input_ids[0], token_type_ids[0], attention_mask[0],
         input_ids[1], token_type_ids[1], attention_mask[1],
         labels)
    
  def __len__(self):
    return len(self.data_dict['input_ids'])

In [33]:
class UnsupCollator(Collator):
  def __init__(self, max_seq_len, tokenizer):
    self.max_seq_len = max_seq_len
    self.tokenizer = tokenizer

  def __call__(self, examples):
    # 根据UnsupAFQMCDataset的getitem 有七个数据
    (ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list,
     ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list,
     labels_list) = list(zip(*examples))
    cur_max_seq_len = max(len(input_ids) for input_ids in ab_input_ids_list)
    max_seq_len = min(cur_max_seq_len, self.max_seq_len)
    # 分批整ab, ba（填充与截断）
    ab_input_ids, ab_token_type_ids, ab_attention_mask, labels = self.pad_and_truncate(ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list, labels_list, max_seq_len)
    ba_input_ids, ba_token_type_ids, ba_attention_mask, labels = self.pad_and_truncate(ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list, labels_list, max_seq_len)


    data_dict = {
        'ab_input_ids':ab_input_ids,
        'ab_token_type_ids':ab_token_type_ids,
        'ab_attention_mask':ab_attention_mask,
        'ba_input_ids':ba_input_ids,
        'ba_token_type_ids':ba_token_type_ids,
        'ba_attention_mask':ba_attention_mask,
        'labels':labels
    }
    return data_dict